In [397]:
import pandas as pd
import geopandas as gpd

## Preprocess SC Election result from Precincts

### Function and declaration

In [398]:
def process_precinct_votes(precinct_gdf, columns_to_use):
    """
    Args:
        precinct_gdf (pd.DataFrame): The DataFrame containing precinct vote data.
        columns_to_use (list): The list of columns to consider for vote calculations.

    Returns:
        pd.DataFrame: The updated DataFrame with calculated vote totals, lean, and filtered party-related columns.
    """
    precinct_gdf['TOT_REP'] = 0
    precinct_gdf['TOT_DEM'] = 0
    precinct_gdf['TOT_VOT'] = 0

    republican_columns = []
    democratic_columns = []

    for col in columns_to_use:
        if len(col) >= 7:
            party_code = col[6]
            if party_code == 'R':  # Republican
                precinct_gdf['TOT_REP'] += precinct_gdf[col]
                republican_columns.append(col)
            elif party_code == 'D':  # Democrat
                precinct_gdf['TOT_DEM'] += precinct_gdf[col]
                democratic_columns.append(col)  

    precinct_gdf['TOT_VOT'] = (
        precinct_gdf['TOT_REP'] +
        precinct_gdf['TOT_DEM']
    )

    precinct_gdf['LEAN'] = precinct_gdf.apply(
        lambda row: 'Unknown' if row['TOT_REP'] == 0 and row['TOT_DEM'] == 0 else
                    ('Republican' if row['TOT_REP'] > row['TOT_DEM'] else
                     ('Democratic' if row['TOT_DEM'] > row['TOT_REP'] else 'Unknown')),
        axis=1
    )
    filtered_columns = ['TOT_REP', 'TOT_DEM', 'TOT_VOT', 'LEAN'] + republican_columns + democratic_columns
    precinct_gdf = precinct_gdf[filtered_columns]

    return precinct_gdf


### Main script

In [399]:
sc_precincts_gdf = gpd.read_file('states/south_carolina/geodata/south_carolina_precincts.geojson')
sc_election_gdf = gpd.read_file('raw/precincts/sc_2022_gen_prec/sc_2022_gen_cong_prec/sc_2022_gen_cong_prec.shp')

In [400]:
print(sc_election_gdf.head())

                             UNIQUE_ID COUNTYFP      County      Precinct  \
0  DORCHESTER-:-BEECH HILL 2-(CONG-01)      035  DORCHESTER  BEECH HILL 2   
1       DORCHESTER-:-CYPRESS-(CONG-01)      035  DORCHESTER       CYPRESS   
2      DORCHESTER-:-DELEMARS-(CONG-01)      035  DORCHESTER      DELEMARS   
3       DORCHESTER-:-GIVHANS-(CONG-01)      035  DORCHESTER       GIVHANS   
4     DORCHESTER-:-GIVHANS 2-(CONG-01)      035  DORCHESTER     GIVHANS 2   

  Prec_Code CONG_DIST  GCON01AODD  GCON01DAND  GCON01OWRI  GCON01RMAC  ...  \
0       084        01          10         286           3         540  ...   
1       072        01          16         479           2         682  ...   
2       036        01           2          69           2          85  ...   
3       035        01           4         144           1         296  ...   
4       094        01           8         189           0         360  ...   

   GCON05GGAI  GCON05OWRI  GCON05RNOR  GCON06DCLY  GCON06OWRI  GCON0

In [401]:
print(sc_election_gdf.head())  

                             UNIQUE_ID COUNTYFP      County      Precinct  \
0  DORCHESTER-:-BEECH HILL 2-(CONG-01)      035  DORCHESTER  BEECH HILL 2   
1       DORCHESTER-:-CYPRESS-(CONG-01)      035  DORCHESTER       CYPRESS   
2      DORCHESTER-:-DELEMARS-(CONG-01)      035  DORCHESTER      DELEMARS   
3       DORCHESTER-:-GIVHANS-(CONG-01)      035  DORCHESTER       GIVHANS   
4     DORCHESTER-:-GIVHANS 2-(CONG-01)      035  DORCHESTER     GIVHANS 2   

  Prec_Code CONG_DIST  GCON01AODD  GCON01DAND  GCON01OWRI  GCON01RMAC  ...  \
0       084        01          10         286           3         540  ...   
1       072        01          16         479           2         682  ...   
2       036        01           2          69           2          85  ...   
3       035        01           4         144           1         296  ...   
4       094        01           8         189           0         360  ...   

   GCON05GGAI  GCON05OWRI  GCON05RNOR  GCON06DCLY  GCON06OWRI  GCON0

In [402]:
columns_to_ignore = ['UNIQUE_ID', 'CONG_DIST','COUNTYFP', 'County', 'Prec_Code', 'Precinct', 'geometry']

In [403]:
columns_to_use = [col for col in sc_election_gdf.columns if col not in columns_to_ignore]

In [404]:
print(columns_to_use)

['GCON01AODD', 'GCON01DAND', 'GCON01OWRI', 'GCON01RMAC', 'GCON02DLAR', 'GCON02OWRI', 'GCON02RWIL', 'GCON03OWRI', 'GCON03RDUN', 'GCON04OWRI', 'GCON04RTIM', 'GCON05DHUN', 'GCON05GGAI', 'GCON05OWRI', 'GCON05RNOR', 'GCON06DCLY', 'GCON06OWRI', 'GCON06RBUC', 'GCON07DSCO', 'GCON07OWRI', 'GCON07RFRY']


In [405]:
sc_election_processed_gdf = process_precinct_votes(sc_election_gdf, columns_to_use)

In [406]:
sc_election_processed_gdf = sc_election_processed_gdf.join(sc_election_gdf[columns_to_ignore])

In [407]:
print(sc_election_processed_gdf[['TOT_REP', 'TOT_DEM', 'TOT_VOT', 'LEAN']].head())

   TOT_REP  TOT_DEM  TOT_VOT        LEAN
0      540      286      826  Republican
1      682      479     1161  Republican
2       85       69      154  Republican
3      296      144      440  Republican
4      360      189      549  Republican


In [408]:
print(sc_election_processed_gdf[['LEAN']].value_counts())

LEAN      
Republican    1672
Democratic     597
Unknown          8
Name: count, dtype: int64


In [409]:
print(sc_election_processed_gdf.columns.values)

['TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'GCON01RMAC' 'GCON02RWIL'
 'GCON03RDUN' 'GCON04RTIM' 'GCON05RNOR' 'GCON06RBUC' 'GCON07RFRY'
 'GCON01DAND' 'GCON02DLAR' 'GCON05DHUN' 'GCON06DCLY' 'GCON07DSCO'
 'UNIQUE_ID' 'CONG_DIST' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct'
 'geometry']


In [138]:
print(sc_election_processed_gdf.columns.values)

['TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'GCON01RMAC' 'GCON02RWIL'
 'GCON03RDUN' 'GCON04RTIM' 'GCON05RNOR' 'GCON06RBUC' 'GCON07RFRY'
 'GCON01DAND' 'GCON02DLAR' 'GCON05DHUN' 'GCON06DCLY' 'GCON07DSCO'
 'UNIQUE_ID' 'CONG_DIST' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct'
 'geometry']


In [411]:
#drop geometry column
sc_election_processed_gdf = sc_election_processed_gdf.drop(columns='geometry')

In [412]:
sc_election_processed_gdf.to_json("states/south_carolina/election/sc_election.json", orient='records')

## MD election preprocessing

In [353]:
md_precincts_gdf = gpd.read_file('states/maryland/geodata/maryland_precincts.geojson')


In [354]:
print(md_precincts_gdf.head())

                     NAME                                           geometry
0  HOWARD PRECINCT 06-001  POLYGON Z ((-76.83025 39.14757 0.00000, -76.83...
1  HOWARD PRECINCT 05-023  POLYGON Z ((-76.89107 39.22616 0.00000, -76.89...
2  HOWARD PRECINCT 05-018  POLYGON Z ((-76.88639 39.20934 0.00000, -76.88...
3  HOWARD PRECINCT 05-017  POLYGON Z ((-76.88670 39.20810 0.00000, -76.88...
4  HOWARD PRECINCT 05-020  POLYGON Z ((-76.91361 39.14963 0.00000, -76.91...


In [355]:
# load csv
md_election_df = pd.read_csv('raw/precincts/HOUSE_precinct_general.csv')

In [356]:
print(md_election_df.head())

                    precinct    office party_detailed party_simplified  \
0  061110097164_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   
1  061110097164_003026019037  US HOUSE     REPUBLICAN       REPUBLICAN   
2  061110097164_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   
3  061110097164_003026019037  US HOUSE     REPUBLICAN       REPUBLICAN   
4  061110097168_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   

           mode  votes county_name  county_fips jurisdiction_name  \
0      ABSENTEE     32     VENTURA       6111.0           VENTURA   
1      ABSENTEE      1     VENTURA       6111.0           VENTURA   
2  NOT ABSENTEE      1     VENTURA       6111.0           VENTURA   
3  NOT ABSENTEE      3     VENTURA       6111.0           VENTURA   
4      ABSENTEE    303     VENTURA       6111.0           VENTURA   

   jurisdiction_fips  ...       state  special writein  state_po state_fips  \
0             6111.0  ...  CALIFORNIA    False   False       

In [357]:
#filter state_po == MD
md_election_df = md_election_df[md_election_df['state_po'] == 'MD']

In [358]:
print(md_election_df.columns.values)

['precinct' 'office' 'party_detailed' 'party_simplified' 'mode' 'votes'
 'county_name' 'county_fips' 'jurisdiction_name' 'jurisdiction_fips'
 'candidate' 'district' 'dataverse' 'year' 'stage' 'state' 'special'
 'writein' 'state_po' 'state_fips' 'state_cen' 'state_ic' 'date'
 'readme_check' 'magnitude']


In [359]:
print(md_election_df['county_name'].value_counts())

county_name
BALTIMORE CITY     6365
MONTGOMERY         5510
PRINCE GEORGE'S    4545
BALTIMORE          4140
ANNE ARUNDEL       2925
HOWARD             2440
FREDERICK          1880
HARFORD            1335
WASHINGTON         1325
ALLEGANY            925
WICOMICO            810
CARROLL             655
CHARLES             645
DORCHESTER          600
ST MARY'S           540
GARRETT             475
CECIL               420
CALVERT             345
SOMERSET            345
WORCESTER           300
TALBOT              180
QUEEN ANNE'S        165
KENT                150
CAROLINE            120
Name: count, dtype: int64


In [360]:
#find when precinct = 06-001
print(md_election_df[md_election_df['precinct'] == '06-001'])

      precinct    office party_detailed party_simplified          mode  votes  \
54456   06-001  US HOUSE            NaN              NaN  2ND ABSENTEE      0   
54457   06-001  US HOUSE            NaN              NaN  2ND ABSENTEE      0   
54458   06-001  US HOUSE            NaN              NaN  2ND ABSENTEE      0   
54459   06-001  US HOUSE            NaN              NaN  2ND ABSENTEE      0   
54460   06-001  US HOUSE            NaN              NaN  2ND ABSENTEE      0   
...        ...       ...            ...              ...           ...    ...   
86111   06-001  US HOUSE     REPUBLICAN       REPUBLICAN  2ND ABSENTEE     79   
86112   06-001  US HOUSE     REPUBLICAN       REPUBLICAN      ABSENTEE     84   
86113   06-001  US HOUSE     REPUBLICAN       REPUBLICAN  EARLY VOTING    569   
86114   06-001  US HOUSE     REPUBLICAN       REPUBLICAN  ELECTION DAY    177   
86115   06-001  US HOUSE     REPUBLICAN       REPUBLICAN   PROVISIONAL     10   

        county_name  county

In [361]:
md_election_df['NAME'] = md_election_df['county_name'] + ' PRECINCT ' + md_election_df['precinct'].astype(str)
columns_order = ['NAME'] + [col for col in md_election_df.columns if col != 'NAME']
md_election_df = md_election_df[columns_order]

In [362]:
print(md_election_df[md_election_df['NAME'] == 'HOWARD PRECINCT 05-023'])

                         NAME precinct    office party_detailed  \
79771  HOWARD PRECINCT 05-023   05-023  US HOUSE            NaN   
79772  HOWARD PRECINCT 05-023   05-023  US HOUSE            NaN   
79773  HOWARD PRECINCT 05-023   05-023  US HOUSE            NaN   
79774  HOWARD PRECINCT 05-023   05-023  US HOUSE            NaN   
79775  HOWARD PRECINCT 05-023   05-023  US HOUSE            NaN   
79776  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79777  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79778  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79779  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79780  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79781  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79782  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79783  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRAT   
79784  HOWARD PRECINCT 05-023   05-023  US HOUSE       DEMOCRA

In [363]:
print(md_election_df.columns.values)

['NAME' 'precinct' 'office' 'party_detailed' 'party_simplified' 'mode'
 'votes' 'county_name' 'county_fips' 'jurisdiction_name'
 'jurisdiction_fips' 'candidate' 'district' 'dataverse' 'year' 'stage'
 'state' 'special' 'writein' 'state_po' 'state_fips' 'state_cen'
 'state_ic' 'date' 'readme_check' 'magnitude']


In [364]:
print(md_election_df[['NAME','office', 'party_detailed', 'candidate', 'writein', 'district', 'votes']].head())


                             NAME    office party_detailed candidate  writein  \
51996    CAROLINE PRECINCT 01-001  US HOUSE            NaN   WRITEIN     True   
51997     CARROLL PRECINCT 01-001  US HOUSE            NaN   WRITEIN     True   
51998       CECIL PRECINCT 01-001  US HOUSE            NaN   WRITEIN     True   
51999  DORCHESTER PRECINCT 01-001  US HOUSE            NaN   WRITEIN     True   
52000        KENT PRECINCT 01-001  US HOUSE            NaN   WRITEIN     True   

       district  votes  
51996         1      0  
51997         1      0  
51998         1      0  
51999         1      0  
52000         1      0  


In [365]:
md_election_df = md_election_df[md_election_df['writein'] != True]

In [366]:
print(md_election_df[['NAME','office', 'party_detailed', 'candidate', 'district', 'votes']].head())

                               NAME    office party_detailed  candidate  \
52046      WICOMICO PRECINCT 01-001  US HOUSE       DEMOCRAT  MIA MASON   
52047    DORCHESTER PRECINCT 01-001  US HOUSE       DEMOCRAT  MIA MASON   
52048      SOMERSET PRECINCT 01-001  US HOUSE       DEMOCRAT  MIA MASON   
52049      CAROLINE PRECINCT 01-001  US HOUSE       DEMOCRAT  MIA MASON   
52050  QUEEN ANNE'S PRECINCT 01-001  US HOUSE       DEMOCRAT  MIA MASON   

       district  votes  
52046         1      6  
52047         1     36  
52048         1     37  
52049         1     43  
52050         1     55  


In [367]:
md_election_df['variable'] = (
    'GCON' + 
    md_election_df['district'].astype(str).str.zfill(2) + 
    md_election_df['party_detailed'].str[0] + 
    md_election_df['candidate'].apply(lambda x: ''.join([name[0] for name in x.split()]))
)

In [368]:
md_election_df_filtered = md_election_df[['NAME', 'precinct', 'county_name', 'county_fips', 'variable', 'votes']]
print(md_election_df_filtered)

                               NAME precinct   county_name  county_fips  \
52046      WICOMICO PRECINCT 01-001   01-001      WICOMICO      24045.0   
52047    DORCHESTER PRECINCT 01-001   01-001    DORCHESTER      24019.0   
52048      SOMERSET PRECINCT 01-001   01-001      SOMERSET      24039.0   
52049      CAROLINE PRECINCT 01-001   01-001      CAROLINE      24011.0   
52050  QUEEN ANNE'S PRECINCT 01-001   01-001  QUEEN ANNE'S      24035.0   
...                             ...      ...           ...          ...   
89131     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89132     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89133     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89134     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89135     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   

         variable  votes  
52046   GCON01DMM      6  
52047   GCON01DMM     36  
52048   GCON01DMM 

In [369]:
md_election_pivoted_df = md_election_df_filtered.pivot_table(
    index=['NAME'], 
    columns='variable',
    values='votes',
    aggfunc='sum'
).fillna(0).reset_index() 


In [370]:
# Check for duplicates
duplicates = md_election_df_filtered.duplicated(subset=['NAME'], keep=False)
if duplicates.any():
    print("Duplicates found:")
    print(md_election_df_filtered[duplicates])
else:
    print("No duplicates found.")


Duplicates found:
                               NAME precinct   county_name  county_fips  \
52046      WICOMICO PRECINCT 01-001   01-001      WICOMICO      24045.0   
52047    DORCHESTER PRECINCT 01-001   01-001    DORCHESTER      24019.0   
52048      SOMERSET PRECINCT 01-001   01-001      SOMERSET      24039.0   
52049      CAROLINE PRECINCT 01-001   01-001      CAROLINE      24011.0   
52050  QUEEN ANNE'S PRECINCT 01-001   01-001  QUEEN ANNE'S      24035.0   
...                             ...      ...           ...          ...   
89131     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89132     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89133     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89134     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   
89135     FREDERICK PRECINCT 26-002   26-002     FREDERICK      24021.0   

         variable  votes  
52046   GCON01DMM      6  
52047   GCON01DMM     36  


In [371]:
print(md_election_pivoted_df)

variable                       NAME  GCON01DMM  GCON01RAH  GCON02DCDR  \
0          ALLEGANY PRECINCT 01-000        0.0        0.0         0.0   
1          ALLEGANY PRECINCT 02-000        0.0        0.0         0.0   
2          ALLEGANY PRECINCT 03-000        0.0        0.0         0.0   
3          ALLEGANY PRECINCT 04-002        0.0        0.0         0.0   
4          ALLEGANY PRECINCT 04-003        0.0        0.0         0.0   
...                             ...        ...        ...         ...   
2030      WORCESTER PRECINCT 05-002      955.0     1592.0         0.0   
2031      WORCESTER PRECINCT 06-001      921.0     1742.0         0.0   
2032      WORCESTER PRECINCT 06-002      250.0      753.0         0.0   
2033      WORCESTER PRECINCT 06-003      430.0     1115.0         0.0   
2034      WORCESTER PRECINCT 07-001     1532.0     3005.0         0.0   

variable  GCON02RJRS  GCON03DJS  GCON03RCA  GCON04DAGB  GCON04RGEM  \
0                0.0        0.0        0.0         0.

In [372]:
print(len(md_election_pivoted_df))

2035


In [373]:
print(len(md_precincts_gdf))

2043


In [374]:
md_precincts_merged_gdf = md_precincts_gdf.merge(
    md_election_pivoted_df, 
    on='NAME', 
    how='left'
)

In [375]:
# columns_to_keep = ['NAME', 'geometry'] + ['GCON01RAH', 'GCON02DCDR', 'GCON03DJS', 'GCON04DAGB', 'GCON05DSHH', 'GCON06DDJT', 'GCON07DKM', 'GCON08DJR']

In [376]:
# md_precincts_merged_gdf = md_precincts_merged_gdf[columns_to_keep]

In [377]:
print(md_precincts_merged_gdf.columns.values)

['NAME' 'geometry' 'GCON01DMM' 'GCON01RAH' 'GCON02DCDR' 'GCON02RJRS'
 'GCON03DJS' 'GCON03RCA' 'GCON04DAGB' 'GCON04RGEM' 'GCON05DSHH'
 'GCON05RCP' 'GCON06DDJT' 'GCON06GGG' 'GCON06RNCP' 'GCON07DKM' 'GCON07RKK'
 'GCON08DJR' 'GCON08RGTC']


In [378]:
# print unique candidate and district
print(md_election_df[['candidate', 'district']].drop_duplicates())

                    candidate  district
52046               MIA MASON         1
52096             ANDY HARRIS         1
56456  CA DUTCH RUPPERSBERGER         2
56461      JOHNNY RAY SALLING         2
59826           JOHN SARBANES         3
59841         CHARLES ANTHONY         3
63581         ANTHONY G BROWN         4
63586      GEORGE E MCDERMOTT         4
67536           STENY H HOYER         5
67551           CHRIS PALOMBI         5
71061           DAVID J TRONE         6
71076            GEORGE GLUCK         6
71106          NEIL C PARROTT         6
77131            KWEISI MFUME         7
77136         KIMBERLY KLACIK         7
85001            JAMIE RASKIN         8
85011     GREGORY THOMAS COLL         8


## Process voting population:

In [379]:
columns_to_ignore = ['NAME', 'geometry']
columns_to_use = [col for col in md_precincts_merged_gdf.columns if col not in columns_to_ignore]

In [380]:
print(columns_to_use)

['GCON01DMM', 'GCON01RAH', 'GCON02DCDR', 'GCON02RJRS', 'GCON03DJS', 'GCON03RCA', 'GCON04DAGB', 'GCON04RGEM', 'GCON05DSHH', 'GCON05RCP', 'GCON06DDJT', 'GCON06GGG', 'GCON06RNCP', 'GCON07DKM', 'GCON07RKK', 'GCON08DJR', 'GCON08RGTC']


In [381]:
md_precincts_processed_df = process_precinct_votes(md_precincts_merged_gdf, columns_to_use)


In [382]:
print(md_precincts_processed_df.head())

   TOT_REP  TOT_DEM  TOT_VOT        LEAN  GCON01RAH  GCON02RJRS  GCON03RCA  \
0    427.0    987.0   1414.0  Democratic        0.0       427.0        0.0   
1    296.0   1006.0   1302.0  Democratic        0.0         0.0        0.0   
2    231.0    817.0   1048.0  Democratic        0.0         0.0        0.0   
3    393.0    985.0   1378.0  Democratic        0.0         0.0        0.0   
4    699.0   1163.0   1862.0  Democratic        0.0         0.0      699.0   

   GCON04RGEM  GCON05RCP  GCON06RNCP  GCON07RKK  GCON08RGTC  GCON01DMM  \
0         0.0        0.0         0.0        0.0         0.0        0.0   
1         0.0        0.0         0.0      296.0         0.0        0.0   
2         0.0        0.0         0.0      231.0         0.0        0.0   
3         0.0        0.0         0.0      393.0         0.0        0.0   
4         0.0        0.0         0.0        0.0         0.0        0.0   

   GCON02DCDR  GCON03DJS  GCON04DAGB  GCON05DSHH  GCON06DDJT  GCON07DKM  \
0       987

In [383]:
# store geojson to dir
md_precincts_processed_df.to_json("states/maryland/election/md_election.json", orient='records')